# Final Project: Honeycomb AKLT as PEPS

Submitted by Yoav Zack, ID 211677398

We will implement the simple PEPS structure which appeared in the theory file, in `julia`. First, some imports:

In [ ]:
using LinearAlgebra, TensorOperations, Plots, Printf, BenchmarkTools
theme(:default)

Next, we define an AKLT Lattice structure, and some a visualization function for it:

In [ ]:
mutable struct LatticeAKLT
    shape::String
    verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}
    bonds::Vector{Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}}
    ψ::Vector{Array{Float64}}

    function LatticeAKLT(shape::String, a::Real, shift::Tuple, σ_boundary::Int64)
        @assert shape ∈ ("hexagon", "triple", "line")
        @assert a > 0 "Bond length `a` must be positive"
        @assert length(shift) == 2 "Lattice shift must be of length 2"
        
        # define verts location for plots
        if shape == "hexagon"
            locs = [a.*(cos(2π/6*ind), sin(2π/6*ind)) for ind in 0:5]
            bonds = [((ind,2), (ind%6+1,1)) for ind in 1:6]
        elseif shape == "triple"
            locs = [(0.0,0.0)]
            append!(locs, [(sin(2π/3*ind), cos(2π/3*ind)) for ind in 0:2])
            bonds = [((1,ind-1),(ind,1)) for ind in 2:4]
        elseif shape == "line"
            num_verts = 9
            locs = [(0.0,0.0)]
            for ind in 1:num_verts-1
                push!(locs, a.*(ind*cos(π/6), (ind%2)sin(π/6)))
            end
            bonds = [((ind,2),(ind+1,1)) for ind in 1:(num_verts-1)]
            bonds[1] = ((1,1),(2,1)) # fix first connection
        end
        
        # add shift to vertex locations and define verts array
        verts = [(ind, loc .+ shift) for (ind, loc) in enumerate(locs)]
        
        # fill in tensors
        ψ = AKLT_PEPS(verts, bonds, σ_boundary)
        
        # create structure
        new(shape, verts, bonds, ψ)
    end
end

In [ ]:
function showlattice(fig::Plots.Plot, lat::LatticeAKLT, dolabel::Bool)
    x = [loc[1] for (ind, loc) in lat.verts]
    y = [loc[2] for (ind, loc) in lat.verts]

    # plot bonds
    for bond in lat.bonds
        inds = [b[1] for b in bond]
        dx = [lat.verts[i][2][1] for i in inds]
        dy = [lat.verts[i][2][2] for i in inds]
        plot!(fig, dx, dy, label="", color="cyan")
    end

    # plot vertices
    scatter!(fig, x, y, aspect_ratio=1, label="", color="red")

    # add numerical labels to the vertices, if needed
    if dolabel
        for (ind, loc) in lat.verts
            annotate!(fig, loc[1], loc[2], ind)
        end
    end
    return fig
end

The tensorsin the definition of the AKLT lattice are defined using the following functions:

In [ ]:
function AKLT_PEPS(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, bonds::Vector{Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}}, σ_boundary::Int64)
    """
    Builds a PEPS tensor array of the AKLT model for a given set of vertices and the bonds between them, assuming a HoneycombLattice structure (i.e. three connections at most)
    """

    @assert σ_boundary ∈ [1,2] "The value of boundary virtual spins must be up (1) or down (2)"
    
     #First, a Θ tensor per site. Θ tensors have 4 indices: σ,a,b,c
    ψ = [projectionOp() for i in 1:length(verts)]

    # multiply the tensors by the bond matrices: antisymmetric spin-1/2
    Σ = zeros(2,2); Σ[1,2] = 1/sqrt(2); Σ[2,1] = -1/sqrt(2);
    for bond in bonds
        (ind, conn) = bond[1]
        if conn == 1
            @tensor B[σ,ap,b,c] := ψ[ind][σ,a,b,c] * Σ[a,ap]
        elseif conn == 2
            @tensor B[σ,a,bp,c] := ψ[ind][σ,a,b,c] * Σ[b,bp]
        elseif conn == 3
            @tensor B[σ,a,b,cp] := ψ[ind][σ,a,b,c] * Σ[c,cp]
        end
        ψ[ind] = B
    end

    # finally, set the boundary legs of each vertex to specific value, if needed
    ψ_out = []
    for (ind, A) in enumerate(ψ)
        c, _ = countneighbors(verts, verts[ind], bonds)
        if c == 0
            push!(ψ_out, A[:,σ_boundary,σ_boundary,σ_boundary])
        elseif c == 1
            push!(ψ_out, A[:,:,σ_boundary,σ_boundary])
        elseif c == 2
            push!(ψ_out, A[:,:,:,σ_boundary])
        elseif c == 3
            push!(ψ_out, A[:,:,:,:])
        end
    end
    return ψ_out
end

In [ ]:
function projectionOp()
    """
    Generates the Θ operators from theory
    """
    Θ = zeros(4,2,2,2)
    for a in 1:2
        for b in 1:2
            for c in 1:2
                # if a+b+c==3 then a==1 && b==1 && c==1. The rest are similar
                Θ[1,a,b,c] = (a+b+c == 3)
                Θ[2,a,b,c] = (a+b+c == 4)/sqrt(3)
                Θ[3,a,b,c] = (a+b+c == 5)/sqrt(3)
                Θ[4,a,b,c] = (a+b+c == 6)
            end
        end
    end
    return Θ
end

In [ ]:
function countneighbors(verts::Vector{Tuple{Int64, Tuple{Float64, Float64}}}, vert::Tuple{Int64, Tuple{Float64, Float64}}, bonds::Vector{Tuple{Tuple{Int64, Int64}, Tuple{Int64, Int64}}})
    """
    Counts the amount of neighbors a vertex `vert` has in the lattice
    """
    c = 0
    vert_bonds = []
    (ind, loc) = vert
    for bond in bonds
        inds = [b[1] for b in bond]
        if ind ∈ inds
            c += 1
            push!(vert_bonds, bond)
        end
    end
    return c, vert_bonds
end

Let's plot some lattices:

In [ ]:
lat = LatticeAKLT("line" ,1, (0.0,0.0), 1)
fig = showlattice(scatter(), lat, false)
lat = LatticeAKLT("hexagon", 1, (2.0,3.0), 1)
fig = showlattice(fig, lat, false)
lat = LatticeAKLT("triple",1, (5,3.0), 1)
fig = showlattice(fig, lat, false)

I did not intend that to look like a face, but it's too funny for me to change it.

Anyways, in the MPS case, we could use SVD to move the orthogonality center to a single site, and then the contraction of all of ther sites will be trivial. Unfortunately, this is not possible with PEPS, since the orthogonality center is not even well defined. So, we will have to perform full contraction each time. For example:

In [ ]:
Sz = diagm([3,1,-1,-3]./2)
lat.shape = "test" # using the previous lattice structure as base
lat.verts = [(1,(0.0,0.0)),(2,(1.0,1.0)),(3,(2.0,2.0))]
lat.bonds = [((1,1),(2,1)) , ((2,2),(3,1))]
lat.ψ = AKLT_PEPS(lat.verts, lat.bonds, 1)
display(showlattice(scatter(), lat, true))

# contacts the tensors fully
@tensor ψ[σ1, σ2, σ3] := lat.ψ[1][σ1,a] * lat.ψ[2][σ2,a,b] * lat.ψ[3][σ3,b]
@tensor Z = ψ[σ1, σ2, σ3] * ψ[σ1, σ2, σ3]
@tensor O1 = ψ[σ1, σ2, σ3] * Sz[σ1,σ1p] * ψ[σ1p, σ2, σ3]
@tensor O2 = ψ[σ1, σ2, σ3] * Sz[σ2,σ2p] * ψ[σ1, σ2p, σ3]
@tensor O3 = ψ[σ1, σ2, σ3] * Sz[σ3,σ3p] * ψ[σ1, σ2, σ3p]

# rescale and output
[O1,O2,O3]./Z

Note that this is not the expected outcome (which is $[1,1/2,1]$), but this is fine: it's just a boundary condition effect, which can be seen clearly by the fact that the total spin on the lattice is $1+0.5+1=2.5$ as expected:

In [ ]:
sum([O1,O2,O3]./Z)

So, let's write a function which takes a lattice and calculates the expectation value for a given site, for the different geometries we created before:

In [ ]:
function expect(lat::LatticeAKLT, Op::Matrix, ind::Int64)
    """
    Calculates the expectatino value of the operator `Op` on site `ind` of lattice `lat`
    """
    N = length(lat.verts)
    @assert ind ∈ 1:N "Invalid site index for expectation value of type " * lat.shape

    # each geometry is contracted differently:
    if lat.shape == "hexagon"
        # contract along the ring
        @tensor ψ2[a,σ1,σ2,c]          := lat.ψ[1][σ1,a,b]       * lat.ψ[2][σ2,b,c]
        @tensor ψ3[a,σ1,σ2,σ3,d]       := ψ2[a,σ1,σ2,c]          * lat.ψ[3][σ3,c,d]
        @tensor ψ4[a,σ1,σ2,σ3,σ4,e]    := ψ3[a,σ1,σ2,σ3,d]       * lat.ψ[4][σ4,d,e]
        @tensor ψ5[a,σ1,σ2,σ3,σ4,σ5,f] := ψ4[a,σ1,σ2,σ3,σ4,e]    * lat.ψ[5][σ5,e,f]
        @tensor ψ[σ1,σ2,σ3,σ4,σ5,σ6]   := ψ5[a,σ1,σ2,σ3,σ4,σ5,f] * lat.ψ[6][σ6,f,a]

        # moves the relevant dimension to be last
        ψ = permutedims(ψ, circshift(1:length(size(ψ)), -ind))

        # calculates expectaion value
        @tensor Z = ψ[σ1,σ2,σ3,σ4,σ5,σ6] * ψ[σ1,σ2,σ3,σ4,σ5,σ6]
        @tensor O = ψ[σ1,σ2,σ3,σ4,σ5,σ6] * Op[σ6,σ6p] * ψ[σ1,σ2,σ3,σ4,σ5,σ6p]
    elseif lat.shape == "triple"
        # contract all arms to center
        @tensor ψ[σ1,σ2,σ3,σ4]  := lat.ψ[1][σ1,b,c,d] * lat.ψ[2][σ2,b] * lat.ψ[3][σ3,c] * lat.ψ[4][σ4,d]

        # moves the relevant dimension to be last
        ψ = permutedims(ψ, circshift(1:length(size(ψ)), -ind))
        
        # calculates expectaion value
        @tensor Z = ψ[σ1,σ2,σ3,σ4] * ψ[σ1,σ2,σ3,σ4]
        @tensor O = ψ[σ1,σ2,σ3,σ4] * Op[σ4,σ4p] * ψ[σ1,σ2,σ3,σ4p]
    elseif lat.shape == "line"
        num_verts = length(lat.verts)
        ψ = lat.ψ[1]
        for indi in 2:num_verts-1
            ψ_old = reshape(ψ, 4^(indi-1), 2)
            ψ = nothing
            @tensor ψ[σi, σj, b] := ψ_old[σi,a] * lat.ψ[indi][σj,a,b]
        end
        ψ = reshape(ψ, 4^(num_verts-1), 2)
        @tensor ψ_final[σrest,σend] := ψ[σrest,a] * lat.ψ[end][σend,a]
        ψ = reshape(ψ_final, [4 for i in 1:num_verts]...)

        # moves the relevant dimension to be last, and extracts it using reshape
        ψ = permutedims(ψ, circshift(1:length(size(ψ)), -ind))
        ψ = reshape(ψ, :, 4)
        
        # calculates expectaion value
        @tensor Z = ψ[σrest, σi] * ψ[σrest, σi]
        @tensor O = ψ[σrest, σi] * Op[σi, σip] * ψ[σrest, σip]
    else
        error("Expectation value calculation not implemented for lattice of type " * lat.shape)
    end
    
    return O/Z
end

and a function which plots the expectation value for each site:

In [ ]:
function mapexpect(fig::Plots.Plot, lat::LatticeAKLT, Op::Matrix, dig::Int64)
    """
    Loops over vertices and calculate the expectation value of `op` on each of them, then plots the result on `fig`
    """
    fig = showlattice(fig, lat, false)
    Olist = []
    for (ind,loc) in lat.verts
        O = expect(lat, Op, ind)
        annotate!(fig, loc[1], loc[2], round(O, digits=dig))
        push!(Olist, (ind, O))
    end
    return Olist, fig
end

We will check those functions on each of the possible lattice variations we have:

In [ ]:
a = 1.0
σ_boundary = 1
figlist = []
for shape in ["hexagon", "triple", "line"]
    lat = LatticeAKLT(shape ,a, (5.0,1.0), σ_boundary)
    Sz = diagm([3,1,-1,-3]./2)
    Olist, fig = mapexpect(plot(), lat, Sz, 2)
    title!(fig, "\"" * shape * "\" lattice, total Sz=" * string(round(sum([O for (_,O) in Olist]), digits=1)))
    push!(figlist, fig)
end
plot(figlist..., layout=(1,3), size=(1200,350))

The most unusual case is the hexagonal case, in which we see no bonudary effects. This can be explained by noticing that all of the vertices are identical, thus the boundary effects even out and the result is equal spread of the spin along the chain. In each of the cases, the boundary effects cancel out and we get the correct total spin $S_z$.